# FusDB Reactor + Relation Overview

This notebook explains how the repository models a reactor using relations. The focus is the Reactor and Relation classes, not the physics details inside each relation function.


## What this repo does

- Loads reactor definitions from YAML files.
- Stores reactor metadata separately from parameter values.
- Uses Sympy relations to fill in missing values, validate explicit ones, and keep symbolic expressions when underdetermined.
- Emits warnings (not hard failures) when explicit values violate relations beyond tolerance.


## Core classes (high level)

**Reactor**
- Holds metadata (id, name, configuration, etc.).
- Stores parameter values in a single `parameters` dict.
- Tracks explicit parameters, defaults, and per-parameter tolerances.
- Runs relation groups in order (geometry, plasma + confinement, power exhaust).

**Relation**
- Encapsulates one equation as a Sympy expression with an output variable.
- Supports constraints (for example, enforce positive values).
- Can solve for one or more variables based on available inputs.

**RelationSystem**
- Applies a set of Relation objects iteratively.
- Seeds explicit values first, then optional defaults.
- Replaces defaults with computed values when possible.
- Emits warnings for explicit values that violate a relation beyond tolerance.


## YAML format (flattened)

Reactor YAMLs are flattened: parameters live at the top level (no sections). Each parameter can be either a scalar or a mapping with `value`, `tol`, and `method`.

Example snippet:

```yaml
id: "GENERIC"
name: "Generic Reactor"
reactor_configuration: "tokamak"
organization: "Example Lab"

R: 3.0
A: 3.0
elongation: 1.8  # alias for kappa
delta_95: 0.2

tau_E:
  value: 0.7
  method: tau_E_iter_ipb98y2
  tol: 0.05
```

Aliases are defined in `registry/allowed_variables.yaml`. If both an alias and canonical name are present, loading raises a clear error.


## Example: a generic reactor (with aliases and defaults)

This example shows how a minimal reactor definition loads into a Reactor object and how computed values are produced.


In [1]:
from pathlib import Path
import textwrap
import tempfile

from fusdb.loader import load_reactor_yaml

yaml_text = textwrap.dedent("""
    id: "GENERIC"
    name: "Generic Reactor"
    reactor_configuration: "tokamak"
    organization: "Example Lab"

    R: 3.0
    A: 3.0
    elongation: 1.8
    delta_95: 0.2
""")

with tempfile.TemporaryDirectory() as tmp:
    path = Path(tmp) / "reactor.yaml"
    path.write_text(yaml_text)
    reactor = load_reactor_yaml(path)

print("a (computed from R/A):", reactor.a)
print("kappa (from alias elongation):", reactor.kappa)
print("squareness (default):", reactor.squareness)
print("explicit parameters:", sorted(reactor.explicit_parameters))


a (computed from R/A): 1.00000000000000
kappa (from alias elongation): 1.80000000000000
squareness (default): 1.00000000000000
explicit parameters: ['A', 'R', 'S_p', 'V_p', 'a', 'delta', 'delta_95', 'kappa', 'kappa_95', 'squareness']


## Underdetermined systems become symbolic

If not enough inputs are provided, the RelationSystem keeps symbolic relations. This lets you keep the reactor definition even when it is underconstrained.

The example below loads only metadata; several geometry values will remain symbolic expressions or equations.


In [2]:
from pathlib import Path
import textwrap
import tempfile
import sympy as sp

from fusdb.loader import load_reactor_yaml

yaml_text = textwrap.dedent("""
    id: "UNDER"
    name: "Underdetermined Reactor"
    reactor_configuration: "tokamak"
    organization: "Example Lab"
""")

with tempfile.TemporaryDirectory() as tmp:
    path = Path(tmp) / "reactor.yaml"
    path.write_text(yaml_text)
    reactor = load_reactor_yaml(path)

print("A (aspect ratio):", reactor.A)
print("R (major radius):", reactor.R)
print("a (minor radius):", reactor.a)
print("A is symbolic:", isinstance(reactor.A, sp.Basic))


A (aspect ratio): A
R (major radius): R
a (minor radius): a
A is symbolic: True


## Relation selection (method overrides)

If a parameter specifies a `method`, the system selects the matching relation by name. The match is case-insensitive and ignores separators (underscores, hyphens).

If a method is not available, a warning is emitted and the default relations for that reactor configuration are used instead.


## Where the behavior lives (code map)

- `src/fusdb/loader.py` reads YAML and builds Reactor inputs.
- `src/fusdb/reactors_class.py` orchestrates relation solving.
- `src/fusdb/relation_class.py` applies relations and emits warnings.
- `src/fusdb/relation_util.py` provides shared Sympy helpers.
- `src/fusdb/reactor_util.py` holds allowed tags, aliases, and relation selection logic.
